In [9]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel
import torch
import pandas as pd
from tqdm import tqdm
from collections import defaultdict, Counter
import numpy as np
from textblob import TextBlob
import pickle

tqdm.pandas()

In [10]:
import dgl
from dgl.data.utils import save_graphs, load_graphs


In [11]:
dataset = 'Snopes'

In [12]:
df_train = pd.read_csv(f'formatted_data/declare/{dataset}/mapped_data/5fold/train_0.tsv', delimiter='\t')
df_test = pd.read_csv(f'formatted_data/declare/{dataset}/mapped_data/5fold/test_0.tsv', delimiter='\t')
df_dev = pd.read_csv(f'formatted_data/declare/{dataset}/mapped_data/dev.tsv', delimiter='\t')

In [13]:
df = pd.concat([df_train, df_test, df_dev])

In [14]:
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli').cuda()
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

def score(sequence1, sequence2):

    # run through model pre-trained on MNLI
    x = tokenizer.encode(sequence1, sequence2, return_tensors='pt',
                      truncation_strategy='only_first')
    logits = nli_model(x.cuda())[0]

    # we throw away "neutral" (dim 1) and take the probability of
    # "entailment" (2) as the probability of the label being true 
    entail_contradiction_logits = logits[:,[0,2]]
    probs = entail_contradiction_logits.softmax(dim=1)
    prob_label_is_true = probs[:,1]
    return prob_label_is_true.detach().item()

In [18]:
df['score'] = df.progress_apply(lambda row : score(row.claim_text, row.evidence), axis=1)

  0%|                                                 | 0/29242 [00:00<?, ?it/s]/home/dunght/anaconda3/envs/new/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2335: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(
100%|█████████████████████████████████████| 29242/29242 [14:52<00:00, 32.76it/s]


In [19]:
embedding_dict = pickle.load(open(f'./embedding_dicts_{dataset}_fold_0.pkl','rb'))


In [49]:
claim_feats, claim_labels, claim_texts = [], [], []
claim_ori_ids = []
claim_ids = {}
claim_labels = []
src_support_claims = defaultdict(set)
src_not_support_claims = defaultdict(set)

for _, record in tqdm(df.iterrows()):
    if record.id_left not in claim_ids:
        claim_texts.append(record.claim_text)
        claim_feats.append(embedding_dict[record.id_left])
        claim_labels.append(int(record.cred_label))
        claim_ids[record.id_left] = len(claim_ids)
        claim_ori_ids.append(record.id_left)
    claim_id = claim_ids[record.id_left]

    if record.score < 0.5:
        src_not_support_claims[record.evidence_source].add(claim_id)
    else:
        src_support_claims[record.evidence_source].add(claim_id)
claim_features = np.array(claim_feats)
claim_labels = np.array(claim_labels)
claim_ori_ids = np.array(claim_ori_ids)

29242it [00:01, 16167.23it/s]


In [50]:
for src in list(src_support_claims):
    src_support_claims[src] = list(src_support_claims[src])
for src in list(src_not_support_claims):
    src_not_support_claims[src] = list(src_support_claims[src])

In [51]:
edges = defaultdict(set)
for src in src_support_claims:
    claims =  src_support_claims[src]
    for i in range(len(claims)):
        for j in range(i + 1, len(claims)):
            edges['1-1'].add((claims[i], claims[j]))
            edges['1-1'].add((claims[j], claims[i]))
for src in src_not_support_claims:
    claims = src_not_support_claims[src]
    for i in range(len(claims)):
        for j in range(i + 1, len(claims)):
            edges['0-0'].add((claims[i], claims[j]))
            edges['0-0'].add((claims[j], claims[i]))
for src in set(src_not_support_claims).intersection(src_support_claims):
    claims_1 = src_support_claims[src]
    claims_0 = src_not_support_claims[src]
    for claim_1 in claims_1:
        for claim_0 in claims_0:
            edges['1-0'].add((claim_1, claim_0))
            edges['0-1'].add((claim_0, claim_1))

In [52]:
for claims_ in src_support_claims.values():
    for i in range(len(claims_)):
        for j in range(i + 1, len(claims_)):
            edges['mutual_evd'].add((claims_[i], claims_[j]))
            edges['mutual_evd'].add((claims_[j], claims_[i]))

In [53]:
term_to_claim_ids = defaultdict(set)
for claim_id, claim_text in tqdm(enumerate(claim_texts)):
    text = TextBlob(claim_text)
    for term in text.noun_phrases:
        term_to_claim_ids[term].add(claim_id)


4341it [00:00, 4950.97it/s]


In [54]:
for claim_ids_ in term_to_claim_ids.values():
    claim_ids_ = list(claim_ids_)
    for i in range(len(claim_ids_)):
        for j in range(i + 1, len(claim_ids_)):
            edges['np'].add((claim_ids_[i], claim_ids_[j]))
            edges['np'].add((claim_ids_[j], claim_ids_[i]))

In [55]:
for etype in list(edges):
    edges[etype] = np.array([list(e) for e in edges[etype]])

In [56]:
for i in tqdm(range(5)):
    df_train = pd.read_csv(f'formatted_data/declare/{dataset}/mapped_data/5fold/train_{i}.tsv', delimiter='\t')
    df_test = pd.read_csv(f'formatted_data/declare/{dataset}/mapped_data/5fold/test_{i}.tsv', delimiter='\t')
    df_dev = pd.read_csv(f'formatted_data/declare/{dataset}/mapped_data/dev.tsv', delimiter='\t')
    train_ids, val_ids, test_ids = set(df_train.id_left), set(df_dev.id_left), set(df_test.id_left)
    train_mask, val_mask, test_mask = [], [], []
    for id_left in claim_ori_ids:
        is_train, is_val, is_test = 0, 0, 0
        if id_left in train_ids:
            is_train = 1
        elif id_left in test_ids:
            is_test = 1
        else:
            is_val = 1
        train_mask.append(is_train)
        val_mask.append(is_val)
        test_mask.append(is_test)
        
    graphs = []
    for ignored_rels in [['mutual_evd'], 
                        ['mutual_evd', 'np'],
                        ['1-1', '0-0', '1-0', '0-1'],
                        ['1-1', '0-0', '1-0', '0-1', 'np']]:
        g = dgl.heterograph({
            ('claim', etype, 'claim') : (merged_edges_[:, 0], merged_edges_[:, 1])
            for etype, merged_edges_ in edges.items() if etype not in ignored_rels
        }, num_nodes_dict = {'claim' : len(claim_labels)}).to('cuda')
        g.ndata['train_mask'] = torch.BoolTensor(train_mask).cuda()
        g.ndata['val_mask'] = torch.BoolTensor(val_mask).cuda()
        g.ndata['test_mask'] = torch.BoolTensor(test_mask).cuda()
        g.ndata['label'] = torch.LongTensor(claim_labels).cuda()
        g.ndata['feature'] = torch.FloatTensor(claim_features).cuda()
        graphs.append(g)
    save_graphs(f"../GAGA/pytorch_gaga/preprocessing/{dataset}_{i}.bin", graphs)

100%|█████████████████████████████████████████████| 5/5 [00:01<00:00,  4.70it/s]


In [59]:
gs = load_graphs('../GAGA/pytorch_gaga/preprocessing/Snopes_0.bin')[0]

In [60]:
for g in gs:
    print(g.etypes)

['0-0', '0-1', '1-0', '1-1', 'np']
['0-0', '0-1', '1-0', '1-1']
['mutual_evd', 'np']
['mutual_evd']
